# L07.1 Softmax Regression with Pytorch
---
이름: 김태산

학과: 자동차공학과

학번: 20143211

---
### Pytorch로 Softmax Regression 구현

In [15]:
import torch
# 학습 데이터 생성
x_train = torch.FloatTensor([ [1,2,1,1], [2,1,3,2], [3,1,3,4], [4,1,5,5], [1,7,5,5],
                             [1,2,5,6], [1,6,6,6], [1,7,7,7] ])
y_train = torch.FloatTensor([ [0,0,1], [0,0,1], [0,0,1], [0,1,0], [0,1,0], [0,1,0],
                             [1,0,0], [1,0,0] ])

In [16]:
# W, b 초기화
W = torch.zeros(4, 3, requires_grad=True)
b = torch.zeros(1, 3, requires_grad=True)

# Optimizer 생성
optimizer = torch.optim.Adam([W, b], lr=0.1)

# 반복횟수 설정
for epoch in range(3001):
  # 가설 및 비용 설정
  hypothesis = torch.softmax(torch.mm(x_train, W) + b, dim=1) # dim 파라미터는 softmax 값이 계산될 차원을 의미한다.
  cost = -torch.mean(torch.sum(y_train * torch.log(hypothesis), dim=1))
  
  # Optimizer를 이용한 경사 계산 및 W, b 업데이트
  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  # 학습이 잘 되는지 확인하기 위한 내용 출력
  if epoch % 300 == 0:
    print("epoch: {}, cost: {:.6f}".format(epoch, cost.item()))

epoch: 0, cost: 1.098612
epoch: 300, cost: 0.105263
epoch: 600, cost: 0.042634
epoch: 900, cost: 0.023111
epoch: 1200, cost: 0.014479
epoch: 1500, cost: 0.009879
epoch: 1800, cost: 0.007124
epoch: 2100, cost: 0.005338
epoch: 2400, cost: 0.004113
epoch: 2700, cost: 0.003236
epoch: 3000, cost: 0.002588


- Pytorch를 이용하여 Softmax regression을 직접 구현해보았다.
- 출력 결과를 보면, 훈련이 진행될수록 cost 값이 줄어드는 것으로 보아 훈련이 올바르게 진행되고 있음을 알 수 있다.

In [41]:
# x가 [1,11,10,9], [1,3,4,3], [1,1,0,1] 일 때, y값은?
x_test = torch.FloatTensor([ [1,11,10,9], [1,3,4,3], [1,1,0,1] ])
test_all = torch.softmax(torch.mm(x_test, W) + b, dim=1)
print(test_all)
print(torch.argmax(test_all, dim=1))

tensor([[1.0000e+00, 5.5163e-19, 7.0149e-38],
        [1.4800e-02, 7.4294e-01, 2.4226e-01],
        [1.2256e-33, 9.0835e-12, 1.0000e+00]], grad_fn=<SoftmaxBackward>)
tensor([0, 1, 2])


- test_all의 출력 결과를 보면 테스트 샘플 별로 softmax 함수를 통해 각 클래스에 대한 확률값이 출력되는 것을 알 수 있다.
- 테스트 샘플로 테스트한 결과 첫번째 샘플(x = [1,11,10,9])은 첫번째 클래스로, 두번째 샘플(x = [1,3,4,3])은 두번째 클래스로, 세번째 샘플(x = [1,1,0,1])은 세번째 클래스로 예측값을 출력한다.

### 조금 더 깔끔하게 Softmax

In [32]:
# torch.nn.functional을 F라는 이름으로 사용하겠다고 선언
# torch.nn.functional 모듈에는 이번 실습에서 사용할 cross_entropy() 뿐만 아니라,
# 다양한 손실함수와 활성함수, 인공신경망을 만드는데 사용되는 함수들을 포함하고 있다.
import torch.nn.functional as F

# y_train 수정
# F.cross_entropy를 사용할 때는 y_train의 값을 클래스 번호로 입력할 수 있다.
y_train = torch.LongTensor([2,2,2,1,1,1,0,0])
y_train

tensor([2, 2, 2, 1, 1, 1, 0, 0])

In [33]:
W = torch.zeros(4, 3, requires_grad=True)
b = torch.zeros(1, 3, requires_grad=True)

optimizer = torch.optim.Adam([W, b], lr=0.1)

for epoch in range(3001):
  # 가설, 비용 수정
  z = torch.mm(x_train, W) + b
  cost = F.cross_entropy(z, y_train) # 주의! F.cross_entropy는 softmax와 cross entropy를 합친 것.
                                     # input 위치(z)에는 N*C 크기의 텐서(N: 데이터 갯수, C: 클래스 갯수)를,
                                     # target 위치(y_train)에는 N 크기의 클래스를 나타내는 텐서를 입력한다.

  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  if epoch % 300 == 0:
    print("epoch: {}, cost: {:.6f}".format(epoch, cost.item()))

epoch: 0, cost: 1.098612
epoch: 300, cost: 0.105263
epoch: 600, cost: 0.042634
epoch: 900, cost: 0.023111
epoch: 1200, cost: 0.014479
epoch: 1500, cost: 0.009879
epoch: 1800, cost: 0.007124
epoch: 2100, cost: 0.005338
epoch: 2400, cost: 0.004113
epoch: 2700, cost: 0.003236
epoch: 3000, cost: 0.002588


- 출력 결과를 보면, 위에서 Softmax regression을 직접 구현한 결과와 동일한 결과를 나타냄을 알 수 있다.

In [35]:
# x가 [1,11,10,9], [1,3,4,3], [1,1,0,1] 일 때, y값은?
x_test = torch.FloatTensor([ [1,11,10,9], [1,3,4,3], [1,1,0,1] ])
z = torch.mm(x_test, W) + b
test_all = torch.softmax(z, dim=1)
print(test_all)
print(torch.argmax(test_all, dim=1))

tensor([[1.0000e+00, 5.5163e-19, 7.0149e-38],
        [1.4800e-02, 7.4294e-01, 2.4226e-01],
        [1.2256e-33, 9.0835e-12, 1.0000e+00]], grad_fn=<SoftmaxBackward>)
tensor([0, 1, 2])


- test_all의 출력 결과를 보면 테스트 샘플 별로 softmax 함수를 통해 각 클래스에 대한 확률값이 출력되는 것을 알 수 있다.
- 테스트 샘플로 테스트한 결과도 마찬가지로 첫번째 샘플(x = [1,11,10,9])은 첫번째 클래스로, 두번째 샘플(x = [1,3,4,3])은 두번째 클래스로, 세번째 샘플(x = [1,1,0,1])은 세번째 클래스로 예측값을 출력한다.

In [36]:
# 어차피 맨날 쓰는 W와 b. nn.Linear로 대체하자!
# nn.Linear: x^Tw + b를 간단히 표현하는 방법
# torch.nn는 그래프 연산을 위한 기본 구성 요소들을 포함하고 있다.
# 예를 들어, torch.nn.Linear는 x^Tw + b과 같은 선형 연산을 수행해준다.
import torch.nn as nn

model = nn.Linear(4, 3) # 첫번째 정수는 입력할 데이터(x_train)의 크기를,
                        # 두번째 정수는 출력할 데이터(클래스 갯수)의 크기를 입력한다.
                        # 세번째 파라미터로 bias가 있는데 default값은 True이다.
                        # bias=False로 지정할 경우 편향(b) 항을 추가하지 않는다.

optimizer = torch.optim.Adam(model.parameters(), lr=1)

for epoch in range(3001):
  z = model(x_train)
  cost = F.cross_entropy(z, y_train)

  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  if epoch % 300 == 0:
    print("epoch: {}, cost: {:.6f}".format(epoch, cost.item()))

epoch: 0, cost: 1.547660
epoch: 300, cost: 0.022227
epoch: 600, cost: 0.008657
epoch: 900, cost: 0.004653
epoch: 1200, cost: 0.002914
epoch: 1500, cost: 0.001991
epoch: 1800, cost: 0.001439
epoch: 2100, cost: 0.001080
epoch: 2400, cost: 0.000833
epoch: 2700, cost: 0.000657
epoch: 3000, cost: 0.000526


- 출력 결과를 보면, 이번에는 학습률을 1로 하였기 때문에 Softmax regression을 직접 구현한 결과와 동일한 결과는 아니지만 여전히 cost 값은 훈련이 진행될수록 줄어드는 것으로 보아 훈련이 올바르게 진행되고 있음을 알 수 있다.

In [38]:
# x가 [1,11,10,9], [1,3,4,3], [1,1,0,1] 일 때, y값은?
x_test = torch.FloatTensor([ [1,11,10,9], [1,3,4,3], [1,1,0,1] ])
z = model(x_test)
test_all = torch.softmax(z, dim=1)
print(test_all)
print(torch.argmax(test_all, dim=1))

tensor([[1.0000e+00, 8.1656e-26, 0.0000e+00],
        [7.4271e-07, 7.0770e-01, 2.9230e-01],
        [1.4013e-45, 5.8471e-15, 1.0000e+00]], grad_fn=<SoftmaxBackward>)
tensor([0, 1, 2])


- test_all의 출력 결과를 보면 테스트 샘플 별로 softmax 함수를 통해 각 클래스에 대한 확률값이 출력되는 것을 알 수 있다.
- 테스트 샘플로 테스트한 결과도 마찬가지로 첫번째 샘플(x = [1,11,10,9])은 첫번째 클래스로, 두번째 샘플(x = [1,3,4,3])은 두번째 클래스로, 세번째 샘플(x = [1,1,0,1])은 세번째 클래스로 예측값을 출력한다.
- 위와 같이 Pytorch의 다양한 함수들을 사용하여 직접 Softmax regression을 구현한 것과 동일한 과정을 좀 더 간단한 방법으로 구현할 수 있었다. 